In [1]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
def get_params_to_update(net, feature_extract):
    params_to_update = net.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return params_to_update

def make_alexnet(out_features, feature_extract = False):
    net = models.alexnet(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_mobilenet_v3(out_features, feature_extract = False):
    net = models.mobilenet_v3_small(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[3].in_features
    net.classifier[3] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size, early_stopping):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,early_stopping = early_stopping,
                                 epochs=epochs, optimizer=optimizer, print_results=True)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net, optimizer = net_generator()
        early_stopping = net_training.EarlyStoppingByAccuracy(patience=10)
        results = train_net_random_dataset_split(net, dataset, epochs,optimizer=optimizer,
                                                 early_stopping=early_stopping, batch_size=batch_size)
        all_results.append(results)
        net_training.print_final_results(results)
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

#transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
transform =  transforms.Compose([
    transforms.Normalize((0, 0, 0), (255, 255, 255)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

def net_generator():
    return make_mobilenet_v3(out_features=len(selected_birds), feature_extract = False)

results = cross_validate_net(net_generator, dataset, repeat=5, epochs=100, batch_size=64)
print_validation_results(results)

Training network 1 ...
Params to learn:
	 features.0.0.weight
	 features.0.1.weight
	 features.0.1.bias
	 features.1.block.0.0.weight
	 features.1.block.0.1.weight
	 features.1.block.0.1.bias
	 features.1.block.1.fc1.weight
	 features.1.block.1.fc1.bias
	 features.1.block.1.fc2.weight
	 features.1.block.1.fc2.bias
	 features.1.block.2.0.weight
	 features.1.block.2.1.weight
	 features.1.block.2.1.bias
	 features.2.block.0.0.weight
	 features.2.block.0.1.weight
	 features.2.block.0.1.bias
	 features.2.block.1.0.weight
	 features.2.block.1.1.weight
	 features.2.block.1.1.bias
	 features.2.block.2.0.weight
	 features.2.block.2.1.weight
	 features.2.block.2.1.bias
	 features.3.block.0.0.weight
	 features.3.block.0.1.weight
	 features.3.block.0.1.bias
	 features.3.block.1.0.weight
	 features.3.block.1.1.weight
	 features.3.block.1.1.bias
	 features.3.block.2.0.weight
	 features.3.block.2.1.weight
	 features.3.block.2.1.bias
	 features.4.block.0.0.weight
	 features.4.block.0.1.weight
	 featur

Epoch 0:
	train loss: 1.946858163558803
	validation loss: 1.3366217727530492, validation accuracy: 76.36986301369863%
	Elapsed time: 0:03:54.749025
Epoch 1:
	train loss: 1.1622855219587453
	validation loss: 0.6951420862380773, validation accuracy: 90.06849315068493%
	Elapsed time: 0:03:57.343509
Epoch 2:
	train loss: 0.6610197734157994
	validation loss: 0.3807565120801534, validation accuracy: 92.8082191780822%
	Elapsed time: 0:03:52.294955
Epoch 3:
	train loss: 0.36555646592511526
	validation loss: 0.24067089206551853, validation accuracy: 95.54794520547945%
	Elapsed time: 0:03:51.422621
Epoch 4:
	train loss: 0.217464461155862
	validation loss: 0.17175892424093533, validation accuracy: 97.26027397260275%
	Elapsed time: 0:03:48.577829
Epoch 5:
	train loss: 0.1320129438186796
	validation loss: 0.13017521961911083, validation accuracy: 97.26027397260275%
	Elapsed time: 0:03:47.559030
Epoch 6:
	train loss: 0.08062052910773668
	validation loss: 0.10550113124390172, validation accuracy: 97.

Epoch 0:
	train loss: 1.9929341233206042
	validation loss: 1.392952665890733, validation accuracy: 72.26027397260275%
	Elapsed time: 0:06:45.671662
Epoch 1:
	train loss: 1.2180432101670096
	validation loss: 0.7676231844784462, validation accuracy: 88.6986301369863%
	Elapsed time: 0:08:25.545917
Epoch 2:
	train loss: 0.6879486283143634
	validation loss: 0.44153117153742544, validation accuracy: 92.46575342465754%
	Elapsed time: 0:08:21.811505
Epoch 3:
	train loss: 0.37251744211026777
	validation loss: 0.2954723514514427, validation accuracy: 95.2054794520548%
	Elapsed time: 0:05:55.861547
Epoch 4:
	train loss: 0.20095478018704246
	validation loss: 0.2036924072324413, validation accuracy: 96.91780821917808%
	Elapsed time: 0:05:04.123045
Epoch 5:
	train loss: 0.12791535485772285
	validation loss: 0.16316360776146796, validation accuracy: 97.94520547945206%
	Elapsed time: 0:04:24.493813
Epoch 6:
	train loss: 0.08070079067252307
	validation loss: 0.13308691559997324, validation accuracy: 98

Epoch 5:
	train loss: 0.12290272970346808
	validation loss: 0.1739437659717586, validation accuracy: 97.26027397260275%
	Elapsed time: 0:05:39.688167
Epoch 6:
	train loss: 0.08184558061138435
	validation loss: 0.1437285109742047, validation accuracy: 97.6027397260274%
	Elapsed time: 0:04:34.248815
Epoch 7:
	train loss: 0.05727545697222852
	validation loss: 0.12375886483143454, validation accuracy: 98.28767123287672%
	Elapsed time: 0:04:27.569147
Epoch 8:
	train loss: 0.039900653409303564
	validation loss: 0.11254085100268664, validation accuracy: 98.28767123287672%
	Elapsed time: 0:04:45.784091
Epoch 9:
	train loss: 0.026639219686567885
	validation loss: 0.10904577823534403, validation accuracy: 98.28767123287672%
	Elapsed time: 0:04:28.771724
Epoch 10:
	train loss: 0.024728202467103767
	validation loss: 0.10391525362860667, validation accuracy: 97.94520547945206%
	Elapsed time: 0:04:23.710611
Epoch 11:
	train loss: 0.016230694463997313
	validation loss: 0.0993338017022773, validation 

Epoch 11:
	train loss: 0.02061658953899585
	validation loss: 0.09717612233880447, validation accuracy: 98.63013698630137%
	Elapsed time: 0:04:21.628353
Epoch 12:
	train loss: 0.012925076935804525
	validation loss: 0.09267537969432466, validation accuracy: 98.28767123287672%
	Elapsed time: 0:04:21.536228
Epoch 13:
	train loss: 0.012185501008203467
	validation loss: 0.08861031916553844, validation accuracy: 98.28767123287672%
	Elapsed time: 0:04:21.996523
Epoch 14:
	train loss: 0.010077191753181954
	validation loss: 0.08698046942279763, validation accuracy: 98.63013698630137%
	Elapsed time: 0:04:22.039663
Epoch 15:
	train loss: 0.010803324421212767
	validation loss: 0.08554843088535413, validation accuracy: 98.63013698630137%
	Elapsed time: 0:04:22.204544
Epoch 16:
	train loss: 0.007335213356448487
	validation loss: 0.08847868340472652, validation accuracy: 98.63013698630137%
	Elapsed time: 0:04:21.913419
Epoch 17:
	train loss: 0.006231150377420373
	validation loss: 0.08688600004127581, 

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/mobilenet_v3_small_pretrained.csv")